In [ ]:
import scanpy as sc
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn.metrics as skm
import colorcet as cc
import sklearn as sk
import sklearn.decomposition as decomp
import sklearn.pipeline as pipe
import sklearn.neighbors as nbr
import sklearn.base as skbase
import sklearn.model_selection as skms
import pickle
import os
import joblib

In [ ]:
os.makedirs(
    "figures",
    exist_ok= True,
)
os.makedirs(
    "pickles",
    exist_ok= True,
)

In [ ]:
p822 = sc.read_h5ad("Alex_Lemonade_portal/SCPCS000490/SCPCL000822_filtered_rna.h5ad")
p822

In [ ]:
sc.pl.violin(
    p822,
    ["sum", "detected", "subsets_mito_percent"],
    multi_panel= True,
    save= "_preproc.pdf"
)

In [ ]:
sns.set_style("whitegrid")
sc.pl.scatter(
    p822,
    "sum",
    "detected",
    color= "subsets_mito_percent",
    color_map= "viridis",
    save= "_sum_vs_detected"
)

In [ ]:
p822.var["gene_symbol"] = p822.var["gene_symbol"].astype("str")
p822.var.loc[p822.var["gene_symbol"] == "nan", "gene_symbol"] = p822.var.loc[p822.var["gene_symbol"] == "nan"].index

In [ ]:
sc.pl.highest_expr_genes(
    p822,
    gene_symbols= "gene_symbol",
    save= True
)

In [ ]:
p822 = p822[:, p822.var["gene_symbol"] != "MALAT1"]
p822.layers["norm"] = sc.pp.normalize_total(
    p822,
    copy= True,
    exclude_highly_expressed= True,
    key_added= "norm_factor",
    layer= "spliced",
).layers["spliced"]

p822.layers["log"] = sc.pp.log1p(
    p822,
    copy= True,
    layer= "norm", 
).layers["norm"]

sc.pp.highly_variable_genes(
    p822,
    n_top_genes= 2000,
    flavor= "seurat_v3"
)

sc.pl.highly_variable_genes(
    p822,
    save= True
)

p822.layers["norm_scaled_genes"] = sc.pp.scale(
    p822,
    copy= True,
    layer= "norm"
).layers["norm"]

In [ ]:
class ScPCA(skbase.TransformerMixin, skbase.BaseEstimator):
    def __init__(self, layer= None, n_comps= None, mask= None):
        self.layer = layer
        self.n_comps = n_comps
        self.mask = mask

    def fit(self, X, y= None):
        return self

    def transform(self, X):
        return sc.pp.pca(
            X,
            n_comps= self.n_comps,
            mask_var= self.mask,
            layer= self.layer,
            copy= True,
        )

class ScNeighbors(skbase.TransformerMixin, skbase.BaseEstimator):
    def __init__(self, n_neighbors= 15, n_pcs= None):
        self.n_neighbors = n_neighbors
        self.n_pcs = n_pcs

    def fit(self, X, y= None):
        return self
        
    def transform(self, X):
        return sc.pp.neighbors(
            X,
            n_neighbors= self.n_neighbors,
            n_pcs= self.n_pcs,
            copy= True,
        )
    
class ScLeiden(skbase.TransformerMixin, skbase.BaseEstimator):
    def __init__(self, resolution= 1):
        self.resolution = resolution

    def fit(self, X, y= None):
        return self

    def transform(self, X):
        return sc.tl.leiden(
            X,
            resolution= self.resolution,
            flavor= "igraph",
            copy= True,
        )

class ScScore(skbase.TransformerMixin, skbase.BaseEstimator):

    def fit(self, X, y= None):
        return self

    def score(estimator, X, y= None, sample_weight= None):
        return skm.silhouette_score(
            X.obsp["distances"],
            metric= "precomputed",
            labels= X.obs["leiden"]
        )

In [ ]:
pca = ScPCA(layer= "norm_scaled_genes", mask= "highly_variable")
neighbors = ScNeighbors()
scleid = ScLeiden()
scscorer = ScScore()
workflow = pipe.make_pipeline(pca, neighbors, scleid, scscorer)
param_grid = {
    "scpca__n_comps": range(25, 35),
    "scneighbors__n_neighbors": range(40, 60),
    "scleiden__resolution": np.linspace(0.1, 2, 10) 
}
X_train, X_test = skms.train_test_split(
    p822,
    test_size= 0.2,
    random_state= 0,
)
kfold = skms.KFold(
    shuffle= True,
    random_state= 0,
)


In [ ]:
workflow.fit(p822)
workflow.score(p822)

In [ ]:

grids = skms.GridSearchCV(
    workflow,
    param_grid= param_grid,
    cv= kfold,
    return_train_score= True
) 

with joblib.parallel_backend("loky"):
    grids.fit(X_train)


In [ ]:
grids.best_params_

In [ ]:
with open("pickles/gridsearch", "wb") as f:
    pickle.dump(grids, f)

In [ ]:
fig, axs = plt.subplots(3)
for ax, param in zip(axs, param_grid.keys()):
    sns.lineplot(x= grids.cv_results_["param_" + param], y= grids.cv_results_["mean_test_score"], ax= ax)
    ax.set_title(param)
fig.tight_layout()

In [ ]:
sc.pp.pca(
    p822,
    layer= "norm_scaled_genes",
    #use_highly_variable= True,
    n_comps= 50
)

sc.pl.pca_variance_ratio(
    p822,
    log= True
)

In [ ]:
sc.pl.pca(
    p822,
    dimensions= [(0,1), (2, 3), (4, 5), (6, 7)],
    ncols= 2,
    color= "subsets_mito_percent"
)

In [ ]:
sc.pp.neighbors(
    p822,
    n_neighbors= 41,
)
sc.tl.umap(
    p822,
)
sc.tl.leiden(
    p822,
    resolution= 0.1
)
skm.silhouette_score(
    p822.obsp["distances"],
    metric= "precomputed",
    labels= p822.obs["leiden"]
)

In [ ]:
sc.pl.umap(
    p822,
    color= [
        "CDH1",
        "leiden", 
    ],
    gene_symbols= "gene_symbol",
    cmap= "viridis",
    palette= cc.glasbey_category10
)

In [ ]:
p822.obsp["distances"]

In [ ]:
skm.silhouette_score(
    p822.obsp["distances"],
    metric= "precomputed",
    labels= p822.obs["leiden"]
)